# Terraform Registry to JSON

### 1. HTTP.get()
### 2. JSON.write()

In [ ]:
using HTTP
using JSON
using Dates

In [ ]:
function SaveJSONToFile(objectType::String, dictionaryOfAllRepos::Dict{String,Dict{String,Any}})
    
    # Create a subfolder of this folder called "terraform-registry"
    subfolderpath = joinpath(pwd(), "TfRegistry2JSON")
    if !isdir(subfolderpath)
        mkdir(subfolderpath)
    end

    # Name the file "(providers || modules)_(today's date).json"
    filename = 
        objectType * "_" * 
        string(Dates.today()) * 
        ".json"
    
    filepath = joinpath(subfolderpath, filename)

    # Write the JSON to the file
    file = open(filepath, "w")
    JSON.print(file, dictionaryOfAllRepos)
    close(file)
end

In [ ]:
function GetAllObjectsFromRegistry(objectType::String)
    
    request = HTTP.get(
        "https://registry.terraform.io/v2/" * 
        objectType * 
        "?page%5Bsize%5D=100")
    
    dictionaryOfAllRegistryObjects = Dict{String, Dict{String,Any}}()

    while true

        # Parse HTTP payload into JSON
        httpGetRegistryObjectsBodyString = String(request.body)
        jsonRepos = JSON.parse(httpGetRegistryObjectsBodyString)

        # Iterate through all JSON objects (max = 100)
        for jsonRepo in jsonRepos["data"]

            # Put the "id" attribute in our subobject
            jsonRepo["attributes"]["id"] = jsonRepo["id"]

            # Remove the prefix "https://github.com/" from the repoKey
            usernameReponame = jsonRepo["attributes"]["source"][19:end]

            # Put the user/repo key --> object
            dictionaryOfAllRegistryObjects[usernameReponame] = jsonRepo["attributes"]
        end

        # Break the infinite loop! 
        if jsonRepos["links"]["next"] == nothing
            break
        end

        # Or Get More! 
        sleep(1)
        request = HTTP.get("https://registry.terraform.io/" * jsonRepos["links"]["next"])
    end

    # Save it all to file
    SaveJSONToFile(objectType, dictionaryOfAllRegistryObjects)
end

In [ ]:
@time GetAllObjectsFromRegistry("providers")

In [ ]:
@time GetAllObjectsFromRegistry("modules")